In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64 # This is the dimensionality of the noise, or 'z' vector
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [3]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

Processing...
Done!


In [4]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [5]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [6]:
# Device setting
D = D.to(device)
G = G.to(device)

In [7]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [8]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0379, g_loss: 4.4747, D(x): 0.99, D(G(z)): 0.03
Epoch [0/200], Step [400/600], d_loss: 0.1863, g_loss: 5.3826, D(x): 0.97, D(G(z)): 0.11
Epoch [0/200], Step [600/600], d_loss: 0.1026, g_loss: 5.0107, D(x): 0.96, D(G(z)): 0.06
Epoch [1/200], Step [200/600], d_loss: 0.0651, g_loss: 4.6639, D(x): 0.98, D(G(z)): 0.04
Epoch [1/200], Step [400/600], d_loss: 0.0474, g_loss: 4.5772, D(x): 0.98, D(G(z)): 0.02
Epoch [1/200], Step [600/600], d_loss: 0.2381, g_loss: 3.9279, D(x): 0.94, D(G(z)): 0.13
Epoch [2/200], Step [200/600], d_loss: 1.3135, g_loss: 3.4576, D(x): 0.78, D(G(z)): 0.47
Epoch [2/200], Step [400/600], d_loss: 0.2178, g_loss: 3.6507, D(x): 0.90, D(G(z)): 0.06
Epoch [2/200], Step [600/600], d_loss: 0.5176, g_loss: 5.0471, D(x): 0.78, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.2007, g_loss: 3.4440, D(x): 0.95, D(G(z)): 0.12
Epoch [3/200], Step [400/600], d_loss: 0.2796, g_loss: 3.6004, D(x): 0.91, D(G(z)): 0.13
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.3449, g_loss: 4.2427, D(x): 0.90, D(G(z)): 0.07
Epoch [31/200], Step [200/600], d_loss: 0.5064, g_loss: 4.4723, D(x): 0.88, D(G(z)): 0.13
Epoch [31/200], Step [400/600], d_loss: 0.2633, g_loss: 6.5400, D(x): 0.87, D(G(z)): 0.02
Epoch [31/200], Step [600/600], d_loss: 0.3575, g_loss: 5.2832, D(x): 0.86, D(G(z)): 0.02
Epoch [32/200], Step [200/600], d_loss: 0.1563, g_loss: 4.8431, D(x): 0.95, D(G(z)): 0.06
Epoch [32/200], Step [400/600], d_loss: 0.2983, g_loss: 3.7420, D(x): 0.89, D(G(z)): 0.08
Epoch [32/200], Step [600/600], d_loss: 0.0976, g_loss: 4.5668, D(x): 0.98, D(G(z)): 0.06
Epoch [33/200], Step [200/600], d_loss: 0.2301, g_loss: 4.8890, D(x): 0.94, D(G(z)): 0.07
Epoch [33/200], Step [400/600], d_loss: 0.3351, g_loss: 3.0402, D(x): 0.91, D(G(z)): 0.12
Epoch [33/200], Step [600/600], d_loss: 0.3982, g_loss: 5.4878, D(x): 0.94, D(G(z)): 0.16
Epoch [34/200], Step [200/600], d_loss: 0.3300, g_loss: 4.6660, D(x): 0.90, D(G(z)): 0.06
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.5809, g_loss: 2.4489, D(x): 0.75, D(G(z)): 0.15
Epoch [61/200], Step [600/600], d_loss: 0.7479, g_loss: 2.6985, D(x): 0.80, D(G(z)): 0.28
Epoch [62/200], Step [200/600], d_loss: 0.8206, g_loss: 1.9858, D(x): 0.73, D(G(z)): 0.22
Epoch [62/200], Step [400/600], d_loss: 0.5581, g_loss: 1.8063, D(x): 0.86, D(G(z)): 0.21
Epoch [62/200], Step [600/600], d_loss: 0.7182, g_loss: 2.4675, D(x): 0.82, D(G(z)): 0.28
Epoch [63/200], Step [200/600], d_loss: 0.5250, g_loss: 2.9170, D(x): 0.80, D(G(z)): 0.17
Epoch [63/200], Step [400/600], d_loss: 0.7025, g_loss: 2.2860, D(x): 0.78, D(G(z)): 0.23
Epoch [63/200], Step [600/600], d_loss: 0.5553, g_loss: 2.4864, D(x): 0.76, D(G(z)): 0.12
Epoch [64/200], Step [200/600], d_loss: 0.6341, g_loss: 2.2177, D(x): 0.83, D(G(z)): 0.21
Epoch [64/200], Step [400/600], d_loss: 0.7488, g_loss: 3.2411, D(x): 0.69, D(G(z)): 0.12
Epoch [64/200], Step [600/600], d_loss: 0.5717, g_loss: 2.1109, D(x): 0.76, D(G(z)): 0.14
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.7091, g_loss: 1.9304, D(x): 0.74, D(G(z)): 0.24
Epoch [92/200], Step [400/600], d_loss: 0.8447, g_loss: 2.0496, D(x): 0.73, D(G(z)): 0.26
Epoch [92/200], Step [600/600], d_loss: 0.8675, g_loss: 1.7138, D(x): 0.74, D(G(z)): 0.29
Epoch [93/200], Step [200/600], d_loss: 0.6215, g_loss: 1.9039, D(x): 0.81, D(G(z)): 0.24
Epoch [93/200], Step [400/600], d_loss: 0.8810, g_loss: 1.3028, D(x): 0.79, D(G(z)): 0.36
Epoch [93/200], Step [600/600], d_loss: 0.8474, g_loss: 1.7906, D(x): 0.68, D(G(z)): 0.23
Epoch [94/200], Step [200/600], d_loss: 0.7999, g_loss: 2.0252, D(x): 0.70, D(G(z)): 0.21
Epoch [94/200], Step [400/600], d_loss: 0.7159, g_loss: 2.5183, D(x): 0.69, D(G(z)): 0.14
Epoch [94/200], Step [600/600], d_loss: 1.2422, g_loss: 1.7806, D(x): 0.65, D(G(z)): 0.34
Epoch [95/200], Step [200/600], d_loss: 0.8268, g_loss: 1.9798, D(x): 0.74, D(G(z)): 0.26
Epoch [95/200], Step [400/600], d_loss: 0.9292, g_loss: 2.1497, D(x): 0.65, D(G(z)): 0.26
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8509, g_loss: 1.9178, D(x): 0.71, D(G(z)): 0.25
Epoch [122/200], Step [600/600], d_loss: 0.7315, g_loss: 2.0078, D(x): 0.75, D(G(z)): 0.27
Epoch [123/200], Step [200/600], d_loss: 1.0162, g_loss: 2.0226, D(x): 0.58, D(G(z)): 0.16
Epoch [123/200], Step [400/600], d_loss: 0.9447, g_loss: 1.6317, D(x): 0.81, D(G(z)): 0.40
Epoch [123/200], Step [600/600], d_loss: 0.8469, g_loss: 1.8157, D(x): 0.72, D(G(z)): 0.25
Epoch [124/200], Step [200/600], d_loss: 0.8358, g_loss: 1.9155, D(x): 0.69, D(G(z)): 0.25
Epoch [124/200], Step [400/600], d_loss: 0.9289, g_loss: 1.8546, D(x): 0.71, D(G(z)): 0.32
Epoch [124/200], Step [600/600], d_loss: 0.8265, g_loss: 1.5448, D(x): 0.77, D(G(z)): 0.34
Epoch [125/200], Step [200/600], d_loss: 0.7565, g_loss: 2.1562, D(x): 0.70, D(G(z)): 0.23
Epoch [125/200], Step [400/600], d_loss: 0.8668, g_loss: 1.1871, D(x): 0.69, D(G(z)): 0.29
Epoch [125/200], Step [600/600], d_loss: 0.8393, g_loss: 1.6053, D(x): 0.80, D(G(z)): 0.36

Epoch [152/200], Step [600/600], d_loss: 0.9142, g_loss: 1.4709, D(x): 0.71, D(G(z)): 0.34
Epoch [153/200], Step [200/600], d_loss: 0.8721, g_loss: 1.7804, D(x): 0.66, D(G(z)): 0.26
Epoch [153/200], Step [400/600], d_loss: 0.9030, g_loss: 1.6606, D(x): 0.66, D(G(z)): 0.26
Epoch [153/200], Step [600/600], d_loss: 0.8670, g_loss: 1.7875, D(x): 0.69, D(G(z)): 0.28
Epoch [154/200], Step [200/600], d_loss: 1.0149, g_loss: 1.5668, D(x): 0.64, D(G(z)): 0.27
Epoch [154/200], Step [400/600], d_loss: 0.9124, g_loss: 1.6760, D(x): 0.71, D(G(z)): 0.33
Epoch [154/200], Step [600/600], d_loss: 0.9348, g_loss: 1.8370, D(x): 0.65, D(G(z)): 0.27
Epoch [155/200], Step [200/600], d_loss: 1.1475, g_loss: 1.7061, D(x): 0.59, D(G(z)): 0.31
Epoch [155/200], Step [400/600], d_loss: 0.9596, g_loss: 1.2489, D(x): 0.72, D(G(z)): 0.36
Epoch [155/200], Step [600/600], d_loss: 0.8064, g_loss: 1.6162, D(x): 0.69, D(G(z)): 0.25
Epoch [156/200], Step [200/600], d_loss: 1.0525, g_loss: 1.2405, D(x): 0.75, D(G(z)): 0.40

Epoch [183/200], Step [200/600], d_loss: 1.0098, g_loss: 1.3255, D(x): 0.71, D(G(z)): 0.36
Epoch [183/200], Step [400/600], d_loss: 0.9597, g_loss: 1.8213, D(x): 0.68, D(G(z)): 0.32
Epoch [183/200], Step [600/600], d_loss: 0.8339, g_loss: 1.8134, D(x): 0.69, D(G(z)): 0.23
Epoch [184/200], Step [200/600], d_loss: 0.9257, g_loss: 1.4600, D(x): 0.67, D(G(z)): 0.30
Epoch [184/200], Step [400/600], d_loss: 1.1104, g_loss: 1.4095, D(x): 0.60, D(G(z)): 0.31
Epoch [184/200], Step [600/600], d_loss: 0.9535, g_loss: 1.2594, D(x): 0.71, D(G(z)): 0.34
Epoch [185/200], Step [200/600], d_loss: 0.7908, g_loss: 1.7941, D(x): 0.69, D(G(z)): 0.23
Epoch [185/200], Step [400/600], d_loss: 0.9186, g_loss: 1.6235, D(x): 0.66, D(G(z)): 0.26
Epoch [185/200], Step [600/600], d_loss: 1.0121, g_loss: 1.4494, D(x): 0.68, D(G(z)): 0.32
Epoch [186/200], Step [200/600], d_loss: 1.1062, g_loss: 1.4187, D(x): 0.70, D(G(z)): 0.41
Epoch [186/200], Step [400/600], d_loss: 0.7799, g_loss: 1.0742, D(x): 0.76, D(G(z)): 0.32